In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
import pandas as pd
from scipy import optimize
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
#define architecture for three waveforms
class Net(nn.Module):                        # model class      
    def __init__(self):                      
        super().__init__()
        self.fc1 = nn.Linear(8, 16)    # four-layered network; number of neurons in each layer
        self.fc2 = nn.Linear(16, 256)   # equals number of output; no. of bias = no. of outputs; weights = no. of(input*output)
        self.fc3 = nn.Linear(256, 16)
        self.fc4 = nn.Linear(16,1)

    def forward(self, x):              # x is the input vector #
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)                # real value
        return x

In [2]:
data = pd.read_excel('./DataSplit/'+Material+'/_Trig/' + Material +'_Trig'+ '.xlsx')

DataF = data['Freq (kHz)'].to_numpy()
DataB = data['Bpk (mT)'].to_numpy()
DataT = data['T'].to_numpy()
DataD = data['Duty'].to_numpy()
DataS1 = data['Pos_dB/dt'].to_numpy()
DataS2 = data['Neg_dB/dt'].to_numpy()
DataIndTrig = data['DataIndex'].to_numpy()

f1 = DataF/DataD
f2 = DataF/(1-DataD)

DataF = np.log10(DataF)
DataB = np.log10(DataB)
DataT = np.log10(DataT)
DataS1 = np.log10(abs(DataS1))
DataS2 = np.log10(abs(DataS2))
f1 = np.log10(f1)
f2 = np.log10(f2)

DataF = DataF.reshape((-1,1))
f1 = f1.reshape((-1,1))
f2 = f2.reshape((-1,1))
DataB = DataB.reshape((-1,1))
DataT = DataT.reshape((-1,1))
DataD = DataD.reshape((-1,1))
DataS1 = DataS1.reshape((-1,1))
DataS2 = DataS2.reshape((-1,1))

input_data = np.concatenate((f1,f2,DataB,DataT,DataD,DataF,DataS1,DataS2),axis=1)
input_tensors = torch.from_numpy(input_data).view(-1,8)
dataset_trig = torch.utils.data.TensorDataset(input_tensors)

C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:12: RuntimeWarning: invalid value encountered in log10
  DataF = np.log10(DataF1)
C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:18: RuntimeWarning: invalid value encountered in log10
  DataTime = np.log10(DataTime1)


In [ ]:
test_loader_trig = torch.utils.data.DataLoader(dataset_trig, batch_size = 128, shuffle = False)

net_trig = Net().double()
ckpt_trig = torch.load('./Trained_Models/'+Material+'_trig_train.pt')
net_trig = ckpt_trig['model'] # load the trained model
y_pred = []
net_trig.eval()
with torch.no_grad():
    for inputs, in test_loader_trig:
        test_out = net_trig(inputs)
        y_pred.append(test_out)

y_pred = torch.cat(y_pred, dim=0)
trig_test_out = (10**(y_pred.numpy()))*1e3
trig_index    = DataIndTrig